In [45]:
import pandas as pd

mergedData = pd.read_csv("data/merged.csv")

mergedData['id'] = mergedData['id'].apply(str)

mergedData.head(10)

,id,content,date,metions,hashtags,in_reply_to_user_id_str,is_retweet,retweets,favorites
0,1698308935,Be sure to tune in and watch Donald Trump on L...,2009-05-04 13:54:25,NaN,NaN,NaN,False,501.0,879.0
1,1701461182,Donald Trump will be appearing on The View tom...,2009-05-04 20:00:10,NaN,NaN,NaN,False,33.0,271.0
2,1737479987,Donald Trump reads Top Ten Financial Tips on L...,2009-05-08 08:38:08,NaN,NaN,NaN,False,12.0,20.0
3,1741160716,New Blog Post: Celebrity Apprentice Finale and...,2009-05-08 15:40:15,NaN,NaN,NaN,False,10.0,27.0
4,1773561338,"""My persona will never be that of a wallflower...",2009-05-12 09:07:28,NaN,NaN,NaN,False,1421.0,1954.0
5,1776419923,"Miss USA Tara Conner will not be fired - ""I've...",2009-05-12 14:21:55,NaN,NaN,NaN,False,28.0,28.0
6,1786560616,Listen to an interview with Donald Trump discu...,2009-05-13 12:38:28,NaN,NaN,NaN,False,14.0,17.0
7,1796477499,"""Strive for wholeness and keep your sense of w...",2009-05-14 11:30:40,NaN,NaN,NaN,False,17.0,27.0
8,1806258917,"Enter the ""Think Like A Champion"" signed book ...",2009-05-15 09:13:13,NaN,NaN,NaN,False,14.0,9.0
9,1820624395,"""When the achiever achieves, it's not a platea...",2009-05-16 17:22:45,NaN,NaN,NaN,False,19.0,49.0


In [62]:
qualityCheckData = pd.read_csv("data/qualityCheckData_2020-06-13.csv")

# change id column to string and make it the index
qualityCheckData['id'] = qualityCheckData['id'].apply(str)
qualityCheckData = qualityCheckData.set_index('id')

qualityCheckData.head(10)

,id_str,text,created_at,retweet_count,favorite_count
id,,,,,
1698308935,https://twitter.com/realDonaldTrump/status/169...,Be sure to tune in and watch Donald Trump on L...,2009-05-04 18:54:25+00:00,509,917
1701461182,https://twitter.com/realDonaldTrump/status/170...,Donald Trump will be appearing on The View tom...,2009-05-05 01:00:10+00:00,34,269
1737479987,https://twitter.com/realDonaldTrump/status/173...,Donald Trump reads Top Ten Financial Tips on L...,2009-05-08 13:38:08+00:00,13,20
1741160716,https://twitter.com/realDonaldTrump/status/174...,New Blog Post: Celebrity Apprentice Finale and...,2009-05-08 20:40:15+00:00,11,27
1773561338,https://twitter.com/realDonaldTrump/status/177...,"""My persona will never be that of a wallflower...",2009-05-12 14:07:28+00:00,1376,1951
1776419923,https://twitter.com/realDonaldTrump/status/177...,"Miss USA Tara Conner will not be fired - ""I've...",2009-05-12 19:21:55+00:00,29,29
1786560616,https://twitter.com/realDonaldTrump/status/178...,Listen to an interview with Donald Trump discu...,2009-05-13 17:38:28+00:00,15,17
1796477499,https://twitter.com/realDonaldTrump/status/179...,"""Strive for wholeness and keep your sense of w...",2009-05-14 16:30:40+00:00,18,28
1806258917,https://twitter.com/realDonaldTrump/status/180...,"Enter the ""Think Like A Champion"" signed book ...",2009-05-15 14:13:13+00:00,15,10


Comparing the content of the merged Data with the content of the data scraped at 2020-06-13.

In [72]:
from difflib import SequenceMatcher

def similarity(a, b):
    # get rid of any whitespaces, tabs, newlines and carriage returns
    # in front of and behind the string, just in case.
    a = a.strip(' \t\n\r')
    b = b.strip(' \t\n\r')
    return SequenceMatcher(None, a, b).ratio()

ids_with_no_text = []
ids_with_low_similarity = []
sum_similarities = 0.0
sum_retweetsDistance = 0
sum_favoritesDistance = 0
num_matches = 0
for index, row in mergedData.iterrows():
    if row['id'] in qualityCheckData.index:
        areStrings = isinstance(row['content'], str) and isinstance(qualityCheckData.at[row['id'], 'text' ], str)
        if not areStrings:
            ids_with_no_text.append(row['id'])
            continue
        tmpSimilarity = similarity( row['content'], qualityCheckData.at[row['id'], 'text' ] )
        if tmpSimilarity < 0.9:
            ids_with_low_similarity.append([row['id'], row['content'], qualityCheckData.at[row['id'], 'text']])
        sum_similarities += tmpSimilarity
        sum_retweetsDistance += abs(row['retweets'] - qualityCheckData.at[row['id'], 'retweet_count' ])
        sum_favoritesDistance += abs(row['favorites'] - qualityCheckData.at[row['id'], 'favorite_count' ])
        num_matches += 1

average_similarity = sum_similarities / num_matches
average_retweetDistance = sum_retweetsDistance / num_matches
average_favoriteDistance = sum_favoritesDistance / num_matches


In [73]:
print("average similarity: ", average_similarity)
print("average retweet distance: ", average_retweetDistance)
print("average favorite distance: ", average_favoriteDistance)
print("Number of ids with similarity below 0.9: ", len(ids_with_low_similarity))
print("Number of tweets without text in the check Data: ", len(ids_with_no_text))

average similarity:  0.9577619979408702
average retweet distance:  614.3543933656105
average favorite distance:  1867.4073630219068
Number of ids with similarity below 0.9:  4746
Number of tweets without text in the check Data:  426


As we can see the two datasets are pretty similar. In the list of ids with similarity below 0.9 we can spot why we have a value of 0.95 and not 1. The Library I used to scrape the data for the quality test does not include URL's into the text body of the tweets. Another difference is that mentions have a white space character inbetween the name and the @-sign.

In [80]:
mergedData.describe().loc['mean']['retweets'] / average_retweetDistance

10.371678574169264

In [81]:
mergedData.describe().loc['mean']['favorites'] / average_favoriteDistance

13.846025249734204